In [ ]:
# 导入必要的函数库
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import serial
from PIL import Image
import pylab as pl
from scipy import ndimage
import scipy.ndimage
import threading
import time
from matplotlib.patches import Polygon

In [ ]:
# 打开摄像头，占用内存大，不轻易运行
capture=cv2.VideoCapture(0)

In [ ]:
# 获取并处理图像
def get_image():
    while True:
        # 开全局变量处理，分理处红绿蓝
        global image, image_red, image_green, image_blue
        # 从摄像头获取图像
        _,image=capture.read()
        image_red=image[:,:,2].astype(np.float32)
        image_green=image[:,:,1].astype(np.float32)
        image_blue=image[:,:,0].astype(np.float32)
        #cv2.imshow('image',image)
        #cv2.waitKey(1)

# 获取图像单独开进程
get_image_threading = threading.Thread(target=get_image)
get_image_threading.start()

In [ ]:
# 拍照，保存一张照片
def take_photo(photo_name):
    global capture
    photo_name=photo_name+'.jpg'
    _,image=capture.read()
    cv2.imwrite(photo_name,image)
    print('take photo success')

In [ ]:
# 保存一张图片
def take_photo(photo_name, photo):
    photo=photo.astype(np.uint8)
    photo_name=photo_name+'.jpg'
    cv2.imwrite(photo_name,image)
    print('save photo success')

In [ ]:
# 展示图片
def show_image(image):
    image_show=image.astype(np.uint8)
    plt.imshow(image_show, cmap='gray')
    plt.axis('off')  # 不显示坐标轴
    plt.show()

In [ ]:
# 模式匹配获取目标位置，这里都统一采用灰度图
# 这个是目标图像
aim_image = cv2.imread('aim.png')
# 这个是目标图像的灰度图
aim_gray = cv2.cvtColor(aim_image, cv2.COLOR_BGR2GRAY)
threshold = 0.8
def get_aim_point(aim_gray,image_gray):
    global aim_center
    try:
        result = cv2.matchTemplate(image_gray, aim_gray, cv2.TM_CCOEFF_NORMED)
        locations = np.where(result >= threshold)
        # 转换为(x, y)坐标
        locations = list(zip(*locations[::-1]))
        # 转换为目标中心点坐标
        aim_center=np.array([int(locations[0][1] + aim_image.shape[0] / 2),int(locations[0][0] + aim_image.shape[1] / 2)])
    except:
        aim_center=aim_center
    return(aim_center)

In [ ]:
# 突出图像中的红色目标
def get_red():
    global image, image_red, image_green, image_blue
    image_red_only=image_red*2-image_blue-image_green
    image_red_only[image_red_only<0]=0
    return image_red_only

In [ ]:
# 突出图像中的绿色目标
def get_green():
    global image, image_red, image_green, image_blue
    image_green_only=image_green*2-image_blue-image_red
    image_green_only[image_green_only<0]=0
    return image_green_only

In [ ]:
# 突出图像中的蓝色目标
def get_blue():
    global image, image_red, image_green, image_blue
    image_blue_only=image_blue*2-image_blue-image_red
    image_blue_only[image_blue_only<0]=0
    return image_blue_only

In [ ]:
# 在某一图像的指定位置添加图案
kernel = np.array([
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
])*255
def draw_image(image_gray_to_draw, kernel, kernel_point):
    try:
        image_gray_to_draw[int(kernel_point[0]-11):int(kernel_point[0]+11),int(kernel_point[1]-11):int(kernel_point[1]+11)]=kernel
    except:
        pass
    return image_gray_to_draw

In [ ]:
# 图像的映射变换
def photo_transform(ad,origin,border_point):
    # 原先的四个角的边缘点
    pts = np.float32([[0,0],[0,origin.shape[1]],[origin.shape[0],origin.shape[1]],[origin.shape[0],0]])
    # 由于变换的特殊性，这里后面的点是列行，反着来的
    border_point_1=np.zeros(np.shape(border_point))
    border_point_1[:,0]=border_point[:,1]
    border_point_1[:,1]=border_point[:,0]
    pts1 = np.float32(border_point_1)
    # 生成最为关键的变换矩阵
    M = cv2.getPerspectiveTransform(pts,pts1)
    # 变换正方形
    square_trans = cv2.warpPerspective(origin,M,(ad.shape[1],ad.shape[0]))
    square_trans = cv2.cvtColor(square_trans, cv2.COLOR_RGB2GRAY)
    _, square_trans = cv2.threshold(square_trans, 127, 255, cv2.THRESH_BINARY)
    return square_trans

In [ ]:
# 这一步用于识别黑色条纹的位置，计算误差后传给stm32
def get_road_error(road):
    # 100:540
    global error,k
    d1=np.argwhere(road[70:100,100:540] < 250)
    dis1=np.mean(d1,axis=0)-220
    d2=np.argwhere(road[250:280,100:540] < 250)
    dis2=np.mean(d2,axis=0)-220
    d3=np.argwhere(road[400:420,100:540] < 250)
    dis3=np.mean(d3,axis=0)-220
    dis=0.2*dis1+0.5*dis2+0.3*dis3
    k=dis[1]
    k=min(max(-500,k),500)
    if k>=0:
        error='E'+'+'+str(int(int(k)%1000/100))+str(int(int(k)%100/10))+str(int(k)%10)
    if k<0:
        error='E'+'-'+str(int(int((-k))%1000/100))+str(int(int((-k))%100/10))+str(int((-k))%10)
    return error,k

In [ ]:
# 找寻线路，找出要巡迹的那一条线
def get_road():
    # 先得到排除其他干扰后的路线
    road_only=get_red()
    # 这里可能是别的颜色的线
    road_only

In [ ]:
# 测试摄像头刷新率
def test_fps():
    time1=time.time()
    for i in range(100):
        _,image=capture.read()
        # 在此加入别视觉处理的函数
    time2=time.time()
    print('fps:',100/(time2-time1))

In [ ]:
# ex_2cross=cv2.imread('q.jpg').astype(np.float32)
# ex_edge=cv2.imread('t.jpg').astype(np.float32)
# show_image(ex1)
# show_image(ex2)
# ex_edge_gray=cv2.cvtColor(ex_edge, cv2.COLOR_BGR2GRAY)
# # 二极化
# _,ex_edge_=cv2.threshold(ex_edge_gray, 70, 255, cv2.THRESH_BINARY)
# show_image(ex_edge_gray)
# ex_2cross_gray=cv2.cvtColor(ex_2cross, cv2.COLOR_BGR2GRAY)
# # 二极化
# _,ex_2cross_gray=cv2.threshold(ex_2cross_gray, 70, 255, cv2.THRESH_BINARY)
# show_image(ex_2cross_gray)